In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-01 03:03:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.18MB/s    in 0.2s    

2021-10-01 03:03:23 (5.18 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [11]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [12]:
df.count()

4751577

In [13]:
dropna_df = df.dropna()
dropna_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [14]:
dropna_df.count()

4750852

In [15]:
from pyspark.sql.functions import *

In [17]:
review_id_df = dropna_df.select('review_id','customer_id','product_id','product_parent','review_date')
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3LI5TRP3YIDQL|   10140119|B00TXH4OLC|     384427924| 2015-08-31|
|R3LGC3EKEG84PX|   27664622|B00B6QXN6U|     831769051| 2015-08-31|
| R9PYL3OYH55QY|   45946560|B001GCZXW6|      14067376| 2015-08-31|
|R3PWBAWUS4NT0Q|   15146326|B000003EK6|     566295619| 2015-08-31|
|R15LYP3O51UU9E|   16794688|B00N1F0BKK|     210426072| 2015-08-31|
|R1AD7L0CC3DSRI|   32203364|B00V7KAO7Q|     360249504| 2015-08-31|
|R32FE8Y45QV434|    1194276|B000094Q4P|      20972283| 2015-08-31|
|R3NM4MZ4XWL43Q|   45813052|B00JMK0P1I|     378049560| 2015-08-31|
|R3H4FXX6Q7I37D|   12795687|B008OW1S3O|     141620702| 2015-08-31|
|R30L5PET7LFFDC|   36673840|B00VI2L3L4|     920557135| 2015-08-31|
|  REFRE1LEKLAF|   49453576|B0000041EV|     683128964| 2015-08-31|
|R3JTJ5EQN74E9H|    3285047|B00005YW4H|     268461622| 2015-08

In [21]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'string')]

In [24]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

review_id_df = review_id_df.withColumn("review_date",col("review_date").cast(DateType()))
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [26]:
products_df = dropna_df.select('product_id','product_title')
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00TXH4OLC|Whatever's for Us...|
|B00B6QXN6U|Same Trailer Diff...|
|B001GCZXW6| Soaring (Jazz Club)|
|B000003EK6|     CARIBBEAN PARTY|
|B00N1F0BKK|         Pain Killer|
|B00V7KAO7Q|A Thoughtiverse U...|
|B000094Q4P|Comfort of Her Wings|
|B00JMK0P1I|Sleep Soundly CD ...|
|B008OW1S3O|     An Awesome Wave|
|B00VI2L3L4|          Blurryface|
|B0000041EV|Mozart For Mother...|
|B00005YW4H|   Come Away with Me|
|B00Q9KEZV0| Tomorrow Is My Turn|
|B00GFXRKHW|       The Outsiders|
|B004L3AQ10|  The Lincoln Lawyer|
|B0000025PM|Bach: Goldberg Va...|
|B00701QUN8|Quiet Riot - Grea...|
|B003UJC2QI|Love Or Money/Lov...|
|B00LGA16J2|Jonas Kaufmann - ...|
|B00ZGVMLLE|        Clockworking|
+----------+--------------------+
only showing top 20 rows



In [28]:
products_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [32]:
vine_df = dropna_df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|
|R3LGC3EKEG84PX|          5|            0|          0|   N|
| R9PYL3OYH55QY|          5|            0|          1|   N|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|
|R15LYP3O51UU9E|          5|            0|          0|   N|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|
|R32FE8Y45QV434|          5|            0|          0|   N|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|
|R30L5PET7LFFDC|          5|            1|          1|   N|
|  REFRE1LEKLAF|          4|            1|          1|   N|
|R3JTJ5EQN74E9H|          5|            0|          0|   N|
|R1W2F091LCOAW5|          5|            0|          0|   N|
| RYUMFQRRB1FNM|          3|            

In [34]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]